# DevOps
https://en.wikipedia.org/wiki/DevOps

https://dev.to/arnu515/create-a-pypi-pip-package-test-it-and-publish-it-using-github-actions-part-2-1o83

Fork https://github.com/restrepo/devops

In Settings
* Webhooks: Add zenodo links to DOIs updates
* Secrets → Actions → Actions secrets → `New repository secret`:
  * __Name__: TESTPYPI_PASSWORD

As an example of one standard Python package we will use `anomalies`. Instead of install it with `pip`, we will review the commands which are implemented in the automatization process. 

### Install
For that, we must can be clone the corresponding repository in your local computer in some directory at a second level from `$HOME`:
```bash
mkdir -p prog
cd prog
#Clone the repo in $HOME/prog
git clone https://github.com/restrepo/anomalies.git
```
From here, be sure that you don't have rhe package already installed
```
pip3 uninstall anomalies
```
with `sudo` if necessary.

The `setup.py` file in the main directory of the package contains the metadata and requirements of the package. This allow for the installation either at the user-level or the system-level. At the user level it is just
```bash
python3 setup.py install --user
```

We can check the installation by returning back to the `$HOME` directory and loading the module from there

```python
cd
python3
>>> import anomalies
>>> exit()
```

### Syntax checking with `flake8`
By returning back to the directory with the repository
```bash
cd prog/anomalies
```
Stop if Python syntax erros or undefined names:
```
flake8 . --ignore=C901 --select=E9,F63,F7,F82
```
Warning otherwise:
```
flake8 . --count --ignore=C901 --exit-zero --max-complexity=10 --max-line-length=127 --statistics
```
For further info in each rule, check the corresponding URL, e.g, for `E741`:

https://www.flake8rules.com/rules/E741.html

### Tests
```
pytest
```

### Dependences in setup.py
> See: https://docs.scipy.org/doc/scipy/dev/toolchain.html#numpy
```python
        install_requires=[
            'numpy==1.16.5; python_version=="3.7"',
            'numpy>=1.16.5; python_version=="3.8"',
            'numpy>=1.16.5; python_version=="3.9"'
        ],
```

## GitHub Actions
In `.github/workflows/python-package.yml` this processes are automated by creating ubuntu containes with specific python version with the installed dependences.

This processes are triggered after each commit

In `.github/workflows/python-publish.yml` the deploy processes in pypi.org are automated by creating the pip package and uploading the pypi official server. For that
1. The `setup.py` file must be updated with the proper release number, e.g: `v0.2.5` and the related commit must pass all the previous tests.
2. The release must be created in GitHub by clicking in the "[Releases](https://github.com/restrepo/anomalies/releases)" link in the main page of the repository. There the button "Draft new release" must be clicked on and the proper tag, e.g: `v0.2.5`, must be introduced with the corresponding "Release title". The green "Publish release" button at the end, will trigger the previous `python-publish.yml` GitHub Action
3. Check that the tags in the the `README.md` are updated with a green status and go to https://pypi.org/project/anomalies/ to check that the release have been updated